In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import joblib 


data = pd.read_csv('mbajk_dataset(3).csv')

data['date'] = pd.to_datetime(data['date'])
data.sort_values(by='date', inplace=True)
data.set_index('date', inplace=True)

data = data[['available_bike_stands']]
print(data.head())
data_array = data.values.astype(float)


scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_array)
joblib.dump(scaler, 'scaler6.pkl')

train_size = len(data_scaled) - 1302
train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]

def create_dataset(dataset, window_size):
    X, y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:i+window_size])
        y.append(dataset[i+window_size])
    return np.array(X), np.array(y)

window_size = 24 # 12 ur = 24 * pol ure
X_train, y_train = create_dataset(train_data, window_size)
X_test, y_test = create_dataset(test_data, window_size)
X_train = X_train.reshape(-1, 1, window_size)
X_test = X_test.reshape(-1, 1, window_size)

def build_model(model_type):
    model = keras.Sequential()
    if model_type == 'SimpleRNN':
        model.add(keras.layers.SimpleRNN(32, return_sequences=True, input_shape=(1, window_size)))
        model.add(keras.layers.SimpleRNN(32))
    elif model_type == 'GRU':
        model.add(keras.layers.GRU(32, return_sequences=True, input_shape=(1, window_size)))
        model.add(keras.layers.GRU(32))
    elif model_type == 'LSTM':
        model.add(keras.layers.LSTM(32, return_sequences=True, input_shape=(1, window_size)))
        model.add(keras.layers.LSTM(32))
    
    model.add(keras.layers.Dense(16, activation='relu'))
    model.add(keras.layers.Dense(1))
    return model

models = {}
histories = {}
predictions = {}




for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
    print(f"Training {model_type} model...")
    model = build_model(model_type)
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(X_train, y_train, epochs=50, verbose=1)
    models[model_type] = model
    histories[model_type] = history
    predictions[model_type] = model.predict(X_test)

best_mae = float('inf')
best_model = None
best_model_type = ''

def inverse_transform(scaler, data):
    return scaler.inverse_transform(data.reshape(-1, 1))

for model_type, model in models.items():
    pred = model.predict(X_test)
    pred_original = inverse_transform(scaler, pred)
    y_test_original = inverse_transform(scaler, y_test)
    mae = mean_absolute_error(y_test_original, pred_original)
    print(f'MAE for {model_type}: {mae}')
    
    if mae < best_mae:
        best_mae = mae
        best_model = model
        best_model_type = model_type

model_filename = f'best_model_{best_model_type}.h5'
best_model.save(model_filename)
print(f'Best model ({best_model_type}) saved as {model_filename}')

if not os.path.exists('reports'):
    os.makedirs('reports')

# Shranjevanje metrik za učne podatke
with open('reports/train_metrics.txt', 'w') as f_train:
    for model_type, history in histories.items():
        mae_train = np.mean(history.history['loss'])  
        f_train.write(f'{model_type} MAE on Training Data: {mae_train}\n')

# Shranjevanje metrik za testne podatke
with open('reports/metrics.txt', 'w') as f_test:
    for model_type, model in models.items():
        pred = model.predict(X_test)
        pred_original = inverse_transform(scaler, pred)
        y_test_original = inverse_transform(scaler, y_test)
        mae_test = mean_absolute_error(y_test_original, pred_original)
        f_test.write(f'{model_type} MAE on Test Data: {mae_test}\n')

                           available_bike_stands
date                                            
2023-06-25 19:07:30+00:00                      8
2023-06-25 19:33:48+00:00                      7
2023-06-25 19:45:44+00:00                      6
2023-06-25 19:55:29+00:00                      8
2023-06-25 20:02:16+00:00                     12
Training SimpleRNN model...
Epoch 1/50
543/543 [==============================] - 2s 1ms/step - loss: 0.0051
Epoch 2/50
543/543 [==============================] - 1s 1ms/step - loss: 0.0032
Epoch 3/50
543/543 [==============================] - 1s 1ms/step - loss: 0.0031
Epoch 4/50
543/543 [==============================] - 1s 1ms/step - loss: 0.0029
Epoch 5/50
543/543 [==============================] - 1s 1ms/step - loss: 0.0029
Epoch 6/50
543/543 [==============================] - 1s 1ms/step - loss: 0.0029
Epoch 7/50
543/543 [==============================] - 1s 1ms/step - loss: 0.0027
Epoch 8/50
543/543 [==============================] - 1s 1ms/s

c:\Users\mitja\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


40/40 [==============================] - 0s 3ms/step
